In [ ]:
import torch
import torchvision.models as models
import tensorrt
import torch_tensorrt

from typing import List
from PIL import Image
import io
import os
import torch
import torchvision.transforms as transforms
import asyncio
import base64
#from pydantic import BaseModel
import torch
import torchvision.models as models
from torch.profiler import profile, ProfilerActivity, record_function
import glob 
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading Densenet121 Pretrained Model
model: torch.nn.Module = models.densenet121(weights = DenseNet121_Weights.DEFAULT).to(device)
model.eval()

In [ ]:

tr = torchvision.transforms.Compose([transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:

# Ensure Dynamic inputs
inputs_bs8 = torch_tensorrt.Input(
                          min_shape=[2, 3, 224, 224],
                          opt_shape=[8, 3, 224, 224],
                          max_shape=[16, 3, 224, 224],
                          dtype = torch.half)


trt_module: torch.fx.GraphModule = torch_tensorrt.compile(model,
                                        **{
                                            "inputs": [
                                                inputs_bs8
                                            ],
                                            "enabled_precisions": {torch.half},
                                            "ir": "dynamo",
                                        })

In [ ]:

inference_input = torch.randn((1, 3, 224, 224)).to(dtype = torch.half, device = device)
output = trt_module.module()(inference_input)

In [ ]:

# Model Saving and Loading with ExportedProgram

# Saving
torch_tensorrt.save(trt_module, "trt.pt2")

# Loading
model = torch.export.load("trt.pt2").module()

In [ ]:
model = torch_tensorrt.load('trt.pt2').module()
model(*inputs)